
This example shows how to use the StrOutputParser to parse the output of a language model into a string. This is useful when you want to extract a specific piece of information from the output.

It is a part of the LangChain framework, which is used for building applications with language models.

Prompt -> llm -> parser

Parser types:
1. StrOutputParser
2. CommaSeparatedListOutputParser
3. JsonOutputParser

In [10]:
import os
import sys
from dotenv import load_dotenv

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from Update_Git import git_add, git_commit, git_push

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_dir, ".env")
load_dotenv(dotenv_path=env_path)

add_file = os.path.join(current_dir, 'Test_3.ipynb')
git_add(add_file)
git_commit('Test 3 updated')
git_push('main')

''

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# llm model
llm = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature=0.5
)

# prompt
def call_string_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assisstant. Give me 3 synonyms for the following word."),
        ("human", "{input}")
    ])

    # Output parser
    parser = StrOutputParser()

    # The parser will convert the output of the LLM to a string
    chain = prompt | llm | parser

    # The output of llm is a AI message, so we need to convert it to a string
    # The outout of the parser is a string

    return chain.invoke("nice")
                        
print(call_string_output_parser())


/Users/taha/Documents/Python_codes/Dev-playground/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


1. Pleasant
2. Kind
3. Agreeable


In [7]:
def call_list_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Give me 3 synonyms for the following word, separated by commas."),
        ("human", "{input}")
    ])

    parser = CommaSeparatedListOutputParser() # Output is a list of strings

    chain = prompt | llm | parser

    return chain.invoke("cancer")

print(call_list_output_parser())

['malignancy', 'carcinoma', 'neoplasm']


In [ ]:
def call_json_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract information from the following phrase.\n Formatting instructuions: {format_instructions}"),
        ("human", "{input}")
    ])

    # make a class with the parameters we want to extract from the LLM output
    class Person(BaseModel):
        name: str = Field(description="The name of the person")
        age: int = Field(description="The age of the person")
    
    # create a parser using the class
    parser = JsonOutputParser(pydantic_object=Person)

    # chain
    chain = prompt | llm | parser

    # print(parser.get_format_instructions())

    return chain.invoke({
        "input": "John is 30 years old.",
         "format_instructions": parser.get_format_instructions()

    })

print(call_json_output_parser())
# Output is a dictionary !!!

{'name': 'John', 'age': 30}


In [9]:
def call_json_output_parser():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract information from the following phrase.\n Formatting instructuions: {format_instructions}"),
        ("human", "{input}")
    ])

    class Food(BaseModel):
        recipie: str = Field(description="The name of the recipie")
        ingredients: list = Field(description="Ingredients")
    
    parser = JsonOutputParser(pydantic_object=Food)

    chain = prompt | llm | parser

    return chain.invoke({
        "input": "The ingredients for ghormesabzi are rice, beans, beef, lime, and herbs.",
         "format_instructions": parser.get_format_instructions()

    })

print(call_json_output_parser())


{'recipie': 'ghormesabzi', 'ingredients': ['rice', 'beans', 'beef', 'lime', 'herbs']}
